In [2]:
import argparse
from helper.models.config import Config
from engine.model_utils import *
from helper.models.unet import *
from helper.models.deeplab_mobilenet import *
from helper.models.nvidia_ade20k import *
from engine.data_setup import *
from engine.model_utils import *
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import NeptuneLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from helper.api_key import *

g:\Universe\Materials\Github Repos\Drainage-detection-topography\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print("=== Starting training ===")
print("=== Creating model === ")
config = Config()
model_name = 'Unet'
# Setting up model
assert model_name in MODEL_NAMES, f"Model {model_name} is not found."
if "Unet" in model_name:
    model = UNet(config, type=model_name)
elif model_name == "Segformer":
    model = NvidiaSegformer(config)
else:
    model = DeepLab(config)

print("=== Setting up dataloader === ")
# Getting dataloader
dataloader = get_dataloader(mode='train', device=config.device, batch_size=config.BATCH_SIZE)

=== Starting training ===
=== Creating model === 
Initialized Unet
Unet initialized with hyperparams: {'num_epochs': 100, 'batch_size': 64, 'opt': 'str', 'crit': 'CrossEntropy', 'learning_rate': 0.0001, 'scheduler': None}
=== Setting up dataloader === 
dataset_plot_light


In [5]:
dataloader